In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
ff = pd.read_csv(r'data.csv')

In [3]:
ff.drop(columns=['Unnamed: 0','Photo','Flag','Club Logo','Real Face','Special'], inplace=True)

In [4]:
ff = ff.dropna(subset = ["Position"]).reset_index().drop(['index'],axis=1)

In [5]:
# ff[ff['Position'].notna()] แสดง ff ที่ position ไม่มี na 

# ff_ch = pd.isnull(ff["Reactions"]) 
# ff[ff_ch]

In [ ]:
nulls = ff.isnull().sum().to_frame()
for c,n in nulls.iterrows():
    print(c, n[0])

In [6]:
GKstat = (ff.GKDiving + ff.GKHandling + ff.GKKicking + ff.GKPositioning + ff.GKReflexes + ff.Reactions) // 5.65

In [7]:
ff.insert(loc=48, column='GK', value=GKstat)

In [8]:
def sumstring(text): #สร้างฟัง์ชั่นแยกข้อความ "45+2" เปลี่ยนเป็น INT
    temp = text.split("+")
    return  "%u" % int(int(temp[0])+int(temp[1]))

In [9]:
for i in ff.loc[:,'LS':'RB']:
    ff[i] = ff[i].apply(lambda x: sumstring(x) if pd.notnull(x)==True else x)

In [ ]:
def processValue(text):
    try:  #ติดปัญหาเงิน 0 ยูโร ใช้ try except 
        text=text[1:] #value*M or K
    
        #assign M or K into var plus
        plus=text[-1]
        
        #keep only integer bf M or K
        result=float(text[:-1]) #value without * M or K
    except (ValueError, TypeError):
        return 0
        
    if plus=='M':
        result=result*(10**6)
    elif plus=='K':
        result=result*(10**3)
    return "%.2f" % result

In [ ]:
ff['Wage'] = ff['Wage'].apply(lambda x: processValue(x))
ff['Value'] = ff['Value'].apply(lambda x: processValue(x))
ff['Release Clause'] = ff['Release Clause'].apply(lambda x: processValue(x))

In [10]:
def hValue(height):
    height = height.split("'")
    height = (float(height[0])*30.48) + (float(height[1])*2.54)
    return "%.2f" % height

In [11]:
def wValue(weight):
    weight = weight.split('lbs')
    weight = (float(weight[0])*0.453592)
    return "%.2f" % weight

In [12]:
ff['Height'] = ff['Height'].apply(lambda x: hValue(x))
ff['Weight'] = ff['Weight'].apply(lambda x: wValue(x))

In [13]:
for n in range(84):
    if ff.iloc[:,n].dtypes == 'float64':
        ff.iloc[:,n] = ff.iloc[:,n].astype(int)
    else:
        continue
    n+=1

In [14]:
ff['Height'] = ff['Height'].astype(float)
ff['Weight'] = ff['Weight'].astype(float)

In [ ]:
ff['Wage'] = ff['Wage'].astype(float)
ff['Value'] = ff['Value'].astype(float)
ff['Release Clause'] = ff['Release Clause'].astype(float)

In [ ]:
pd.options.display.max_columns = None

In [15]:
def yearCT(text):
    try:
        y = text.split(", ")
        y = y[1]
    except (IndexError, AttributeError):
        return text
    return y

In [16]:
ff['Contract Valid Until'] = ff['Contract Valid Until'].apply(lambda x: yearCT(x))

In [17]:
ff.iloc[:,22:48] = ff.iloc[:,22:48].fillna(0)

In [18]:
ff.to_csv('ff.csv', index=False, encoding="utf-8-sig")

In [19]:
conn = sqlite3.connect('FIFA_19.db')

In [20]:
ff.columns = ff.columns.str.replace(' ','_')

In [21]:
ff.to_sql('FIFAinfo',con=conn, index=None)